https://colab.research.google.com/github/patrickvonplaten/notebooks/blob/master/Fine_tuning_Wav2Vec2_for_English_ASR.ipynb#scrollTo=KuUbPW7oV-B5

- Wav2Vec2 was pretrained on the audio data of LibriSpeech and LibriVox which both were sampling with 16kHz
- https://github.com/pytorch/fairseq/blob/fcca32258c8e8bcc9f9890bf4714fa2f96b6b3e1/fairseq/models/wav2vec/wav2vec2_asr.py
- https://github.com/pytorch/fairseq/issues/2917
- https://github.com/vrindaprabhu/datasets/blob/186a657263ee07b8583cacd7f6b4a1082ae829d4/datasets/timit_asr/timit_asr.py

In [1]:
import sys
sys.path.append("../")
%load_ext autoreload
%autoreload 2

In [2]:
%env DATASET_PATH=/shared/g-luo/vctk

env: DATASET_PATH=/shared/g-luo/vctk


In [3]:
from data_utils import VCTK
from configs.tacotron2 import DataParams

vctk = VCTK(params=DataParams)

In [11]:
vctk[0]

torch.Size([1, 98473])


{'mfcc': tensor([[  1.8686,   2.0159,   2.0142,  ...,   4.4464,   4.7661,   4.9448],
         [  0.8460,   1.1459,   1.1963,  ...,   3.5572,   3.8828,   4.0570],
         [ -1.1821,  -0.4579,  -0.2986,  ...,   1.8876,   2.2227,   2.3790],
         ...,
         [-10.2131, -10.2689, -11.1016,  ..., -11.3992, -10.7348,  -9.6906],
         [-10.4418, -10.6119, -11.2078,  ..., -11.3348, -10.5025,  -9.5473],
         [-10.2895, -10.3778, -10.9103,  ..., -10.9279, -10.0331,  -9.6225]]),
 'text_tensor': tensor([53, 49, 42, 38, 56, 42, 11, 40, 38, 49, 49, 11, 56, 57, 42, 49, 49, 38,
          7, 11], dtype=torch.int32),
 'text': 'Please call Stella.\n',
 'waveform': tensor([-0.0056, -0.0058, -0.0057,  ..., -0.0300, -0.0290, -0.0313]),
 'speaker_id': 'p225',
 'utterance_id': '001',
 'accent': 'English',
 'gender': 'F'}

In [5]:
# %%capture output
device="cuda"
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

model_name = "facebook/wav2vec2-base"
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name)

Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForCTC: ['quantizer.codevectors', 'project_hid.weight', 'quantizer.weight_proj.weight', 'project_q.bias', 'quantizer.weight_proj.bias', 'project_hid.bias', 'project_q.weight']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [6]:
processor.tokenizer.pad_token_id

0

In [ ]:
# from transformers import TrainingArguments

# training_args = TrainingArguments(
#   output_dir="./runs",
#   group_by_length=True,
#   per_device_train_batch_size=32,
#   evaluation_strategy="steps",
#   num_train_epochs=30,
#   fp16=True,
#   gradient_checkpointing=True,
#   save_steps=500,
#   eval_steps=500,
#   logging_steps=500,
#   learning_rate=1e-4,
#   weight_decay=0.005,
#   warmup_steps=1000,
#   save_total_limit=2,
#   push_to_hub=False,
# )

In [ ]:
# from transformers import Trainer
# from torch.utils.data import Subset

# trainer = Trainer(
#     model=model,
#     data_collator=data_collator,
#     args=training_args,
#     compute_metrics=lambda pred: compute_metrics(pred, processor, wer_metric),
#     train_dataset=train_dataset,
#     eval_dataset=train_dataset,
#     tokenizer=processor.feature_extractor,
# )

Scratchbook

In [ ]:
from data_utils import VCTK

DATASET_PATH="/shared/g-luo/vctk"
vctk = VCTK(path=DATASET_PATH, sample_rate=16000)

audio = vctk.__getitem__(0)["waveform"]
rate = 16000
input_values = processor(audio[0], sampling_rate = rate, return_tensors = 'pt').input_values
input_values = input_values.to(device)

In [ ]:
import torch
logits = model(input_values).logits
predicted_ids = torch.argmax(logits, dim =-1)
transcriptions = tokenizer.decode(predicted_ids[0])
print(transcriptions)
print(vctk.__getitem__(0)["text"])

In [ ]:
# import IPython.display as ipd
# import numpy as np
# import random

# ipd.Audio(data=np.asarray(), autoplay=True, rate=16000)

In [32]:
from wav2vec_asr import Wav2VecASR
model = Wav2VecASR.from_pretrained(
    "facebook/wav2vec2-base", 
    ctc_loss_reduction="mean", 
    # pad_token_id=processor.tokenizer.pad_token_id,
)

Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2VecASR: ['project_q.weight', 'project_hid.weight', 'project_hid.bias', 'quantizer.weight_proj.bias', 'quantizer.codevectors', 'quantizer.weight_proj.weight', 'project_q.bias']
- This IS expected if you are initializing Wav2VecASR from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2VecASR from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2VecASR were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inferenc

In [7]:
from datasets import load_dataset, load_metric

timit = load_dataset("timit_asr")

Reusing dataset timit_asr (/home/g-luo/.cache/huggingface/datasets/timit_asr/clean/2.0.1/5bebea6cd9df0fc2c8c871250de23293a94c1dc49324182b330b6759ae6718f8)


  0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

timit = timit.map(prepare_dataset, remove_columns=timit.column_names["train"], num_proc=4)

Loading cached processed dataset at /home/g-luo/.cache/huggingface/datasets/timit_asr/clean/2.0.1/5bebea6cd9df0fc2c8c871250de23293a94c1dc49324182b330b6759ae6718f8/cache-66e3a43768ef8692.arrow
Loading cached processed dataset at /home/g-luo/.cache/huggingface/datasets/timit_asr/clean/2.0.1/5bebea6cd9df0fc2c8c871250de23293a94c1dc49324182b330b6759ae6718f8/cache-deca7da3d948d9b7.arrow
Loading cached processed dataset at /home/g-luo/.cache/huggingface/datasets/timit_asr/clean/2.0.1/5bebea6cd9df0fc2c8c871250de23293a94c1dc49324182b330b6759ae6718f8/cache-5f47b3a79a168a93.arrow
Loading cached processed dataset at /home/g-luo/.cache/huggingface/datasets/timit_asr/clean/2.0.1/5bebea6cd9df0fc2c8c871250de23293a94c1dc49324182b330b6759ae6718f8/cache-85c3da5dd3a9229a.arrow
Loading cached processed dataset at /home/g-luo/.cache/huggingface/datasets/timit_asr/clean/2.0.1/5bebea6cd9df0fc2c8c871250de23293a94c1dc49324182b330b6759ae6718f8/cache-f946af0806b3a0c8.arrow
Loading cached processed dataset at /hom

In [11]:
from torch.utils.data import DataLoader
loader = DataLoader(vctk, collate_fn=data_collator)

In [43]:
import numpy as np
wer_metric = load_metric("wer")
def compute_metrics(pred):
    pred_logits = pred["logits"].detach().numpy()
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred["label_ids"][pred["label_ids"] == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred["label_ids"], group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [46]:
for i, batch in enumerate(loader):
    print(batch)
    print(model(**batch))
    print(compute_metrics(model(**batch)))
    if i == 1:
        break

{'input_values': tensor([[-0.0847, -0.0867, -0.0863,  ..., -0.3972, -0.3843, -0.4140]]), 'labels': tensor([[23,  3,  3,  3,  3,  3,  4,  3,  3,  3,  3,  4, 12,  3,  3,  3,  3,  3,
          3,  3]])}
{'loss': tensor(47.1158, grad_fn=<MeanBackward0>), 'logits': tensor([[[-0.1050, -0.1237,  0.3164,  ...,  0.2016, -0.2388,  0.1466],
         [-0.0985, -0.1073,  0.2290,  ...,  0.2452, -0.2039,  0.0261],
         [-0.0779, -0.1130,  0.2029,  ...,  0.2275, -0.2271,  0.0281],
         ...,
         [-0.0864,  0.0339,  0.1583,  ...,  0.1883, -0.3124,  0.0421],
         [ 0.0112,  0.0159,  0.0154,  ...,  0.1704, -0.2783,  0.0886],
         [ 0.0784, -0.0262, -0.0360,  ...,  0.1693, -0.2671,  0.1901]]],
       grad_fn=<AddBackward0>), 'label_ids': tensor([[23,  3,  3,  3,  3,  3,  4,  3,  3,  3,  3,  4, 12,  3,  3,  3,  3,  3,
          3,  3]])}
{'wer': 1.0}
{'input_values': tensor([[-0.0553, -0.0553, -0.0591,  ..., -0.0746, -0.0703, -0.0776]]), 'labels': tensor([[7, 3, 3, 4, 3, 3, 3, 4, 3, 3, 

In [3]:
from train import load_data, train
from configs.wav2vec_asr import TrainingParams, ModelParams, DataParams

train_loader, val, collate_fn = load_data(TrainingParams, DataParams)

/home/g-luo/anaconda3/envs/ee225d/lib/python3.9/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [5]:
train(TrainingParams, ModelParams, DataParams, train_loader, val, collate_fn)

/home/g-luo/anaconda3/envs/ee225d/lib/python3.9/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Problem at: /home/g-luo/remote_home/other/ee225d/accent-embeddings/notebooks/../train.py 44 train


KeyboardInterrupt: 

In [ ]:
# Ensure that weights are being saved, memory is ok, wandb, params.model_path